# Time Series Prediction With Deep Learning in Keras

https://machinelearningmastery.com/time-series-prediction-with-deep-learning-in-python-with-keras/

https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv

In [ ]:
import pandas as pd
import numpy as np
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import tensorflow as tf
# tf.enable_eager_execution()
# ValueError: Only TF native optimizers are supported in Eager mode.

In [ ]:
df = pd.read_csv('airline-passengers.csv', usecols=[1])

In [ ]:
train_size = int(len(df) * 0.67)
test_size = len(df) - train_size
train, test = df[:train_size], df[train_size:]
print(len(train), len(test))

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset.iloc[i:(i+look_back), 0].to_numpy()
		dataX.append(a)
		dataY.append(dataset.iloc[i + look_back, 0])
	return np.array(dataX), np.array(dataY).reshape(-1, 1)

In [ ]:
# reshape into X=t and Y=t+1
look_back = 5
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
# create and fit Multilayer Perceptron model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(8, input_dim=look_back, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(trainX, trainY, epochs=200, batch_size=2, verbose=2)

In [ ]:
model.summary()

In [ ]:
import math
# Estimate model performance
trainScore = model.evaluate(trainX, trainY, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore, math.sqrt(trainScore)))
testScore = model.evaluate(testX, testY, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore, math.sqrt(testScore)))

In [ ]:
%matplotlib notebook
import numpy
dataset = df
import matplotlib.pyplot as plt

# generate predictions for training
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(dataset)
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

In [ ]:
plt.plot(np.concatenate([trainPredict, testPredict]), label='pred')
plt.plot(df.iloc[:, 0], label='true')
plt.legend()
plt.show()